In [1]:
# Knights Move game
# Author: Dr. Santiago Enrique Conant Pablos
# Last modification: September 2019

import random
from games import (Game, infinity)
from collections import namedtuple
GameState = namedtuple('GameState', 'to_move, knights, moves')

# ______________________________________________________________________________
# Alpha-Beta Minimax Search

def alphabeta_search(state, game, d=4, cutoff_test=None, eval_fn=None):
    """Search game to determine best action; use alpha-beta pruning.
    This version cuts off search and uses an evaluation function."""

    player = game.to_move(state)

    # Functions used by alphabeta
    def max_value(state, alpha, beta, depth):
        #print('max_value')
        #game.display(state)
        if cutoff_test(state, depth):
            v = eval_fn(state, player)
            #print('max:',v)
            return v
        v = -infinity
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a),
                                 alpha, beta, depth + 1))
            if v >= beta:
                #print('max:',v)
                return v
            alpha = max(alpha, v)
        #print('max:',v)
        return v

    def min_value(state, alpha, beta, depth):
        #print('min_value')
        #game.display(state)
        if cutoff_test(state, depth):
            v = eval_fn(state, player)
            #print('min:',v)
            return v
        v = infinity
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a),
                                 alpha, beta, depth + 1))
            #print("v=",v)
            if v <= alpha:
                #print('min:',v)
                return v
            beta = min(beta, v)
        #print('min:',v)
        return v

    # Body of alphabeta_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or
                   (lambda state, depth: depth == d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -infinity
    beta = infinity
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

#_______________________________________________________________________________
# Auxiliary functions

class Knights_Move(Game):
    """Plays Knights on an N x N . A state contains information about the
       next player to play (to_move), the knight location (knight_loc), and a list
       of possible movements in the form of a list of (row, col) locations."""

    def __init__(self, N=8):
        self.N = N
        moves = [(x, y) for x in range(1, N+1) for y in range(1, N+1)]
        knight1 = random.choice(moves)
        moves.remove(knight1)
        knight2 = random.choice(moves)
        moves.remove(knight2)
        # Always starts the player 1 followed by player 2
        self.initial = GameState(to_move=1, knights={1:knight1, 2:knight2},
                                 moves=set(moves))

    def actions(self, state):
        """The legal movements are the locations generated with chess knight
        movements that are in the possible moves list."""
        return self.__legal_moves(state.moves, state.knights[state.to_move])

    def __legal_moves(self, smoves, kloc):
        """Determines the legal movements of a player on a board."""
        moves = []
        nloc = (kloc[0] + 2, kloc[1] - 1)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] + 2, kloc[1] + 1)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] + 1, kloc[1] + 2)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] - 1, kloc[1] + 2)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] - 2, kloc[1] + 1)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] - 2, kloc[1] - 1)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] - 1, kloc[1] - 2)
        if nloc in smoves: moves.append(nloc)
        nloc = (kloc[0] + 1, kloc[1] - 2)
        if nloc in smoves: moves.append(nloc)
        return moves

    def result(self, state, move):
        """Modifies the current state by changing the knight location"""
        moves = state.moves.copy()
        moves.remove(move)
        player = state.to_move
        knights = state.knights.copy()
        knights[player]=move
        return GameState(to_move=(2 if player == 1 else 1),
                         knights=knights, moves=moves)
                    
    def utility(self, state, player):
        "Return the value to player; 1 for win, -1 for loss, 0 otherwise."
        return 1 if player != state.to_move else -1

    def display(self, state):
        """Displays the state of the game board"""
        print('\\', end=' ')
        for y in range(1, self.N+1): print(y, end=' ')
        print('/')
        for r in range(1, self.N+1):
            print(r, end=' ')
            for c in range(1, self.N+1):
                if state.knights[1]  == (r, c):
                    print(1, end=' ')
                elif state.knights[2]  == (r, c):
                    print(2, end=' ')
                elif (r, c) in state.moves:
                    print('.', end=' ')
                else:
                    print('_', end=' ')
            print(r)
        print('/', end=' ')
        for y in range(1, self.N+1): print(y, end=' ')
        print('\\')


# ______________________________________________________________________________
# evaluation function

def eval_fn_p(state, player):
    """A very simple evaluation function"""
    evaluation = 1
    #print(state.to_move, player)
    if player == 1:
        return evaluation
    else:
        return -evaluation
    
# ______________________________________________________________________________ 
# Our evaluation function       
'''(1.4)(# of possible movements at the to_move site) + 
( if to_move site == to_move site of the opponent, +2) + 
(if proximity to the opponent: less than 3 squares or more than 3 squares, -0.3 per square; 
exactly 3 squares, +1) '''


# ----------------> What it needs to run <--------------------------------------

def actions(state):
    """The legal movements are the locations generated with chess knight
    movements that are in the possible moves list."""
    return legal_moves(state.moves, state.knights[state.to_move])

def legal_moves(smoves, kloc):
    """Determines the legal movements of a player on a board."""
    moves = []
    nloc = (kloc[0] + 2, kloc[1] - 1)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] + 2, kloc[1] + 1)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] + 1, kloc[1] + 2)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] - 1, kloc[1] + 2)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] - 2, kloc[1] + 1)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] - 2, kloc[1] - 1)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] - 1, kloc[1] - 2)
    if nloc in smoves: moves.append(nloc)
    nloc = (kloc[0] + 1, kloc[1] - 2)
    if nloc in smoves: moves.append(nloc)
    return moves

def result(state, move):
    """Modifies the current state by changing the knight location"""
    moves = state.moves.copy()
    moves.remove(move)
    player = state.to_move
    knights = state.knights.copy()
    knights[player] = move
    return GameState(to_move = (2 if player == 1 else 1), knights = knights, moves = moves)

def eval_fn(state, player):

    evaluation = 0
    num_moves = 0

    for a in actions(state):
        new_state = result(state, a)
        for b in actions(new_state):
            if a == b:
                evaluation += 2

    for action in actions(state):
        new_state = result(state, action)
        num_moves_i = len(legal_moves(state.moves, new_state.knights[player]))
        if num_moves_i > num_moves:
            num_moves = num_moves_i
    evaluation += 0.7 * num_moves

    #if player == 1:
    mtn_dist = abs(state.knights[player][0] - state.knights[2][0]) + abs(state.knights[player][1] - state.knights[2][1])
    if 3 < mtn_dist:
        evaluation += (mtn_dist - 3) * (-0.3)
    elif 3 > mtn_dist:
        evaluation += mtn_dist * (-0.3)
    elif 3 == mtn_dist:
        evaluation += 1
    
    print('Evaluation 1: ', evaluation)
    return evaluation

# ______________________________________________________________________________ 
# Our other evaluation function       

def eval_fn_2(state, player):

    evaluation = 0
    num_moves = 0

    for a in actions(state):
        new_state = result(state, a)
        for b in actions(new_state):
            if a == b:
                evaluation += 2

    for action in actions(state):
        new_state = result(state, action)
        num_moves_i = len(legal_moves(state.moves, new_state.knights[player]))
        if num_moves_i > num_moves:
            num_moves = num_moves_i
    evaluation += 0.7 * num_moves
    
    if player == 2:
        blocked_moves = 5*(8 - len(legal_moves(state.moves, new_state.knights[player])))
        evaluation += blocked_moves

    #if player == 1:
    mtn_dist = abs(state.knights[player][0] - state.knights[2][0]) + abs(state.knights[player][1] - state.knights[2][1])
    if 3 < mtn_dist:
        evaluation += (mtn_dist - 3) * (-0.3)
    elif 3 > mtn_dist:
        evaluation += mtn_dist * (-0.3)
    elif 3 == mtn_dist:
        evaluation += 1
    
    print('Evaluation 2: ', evaluation)
    return evaluation

# __________________________________________


# ______________________________________________________________________________
# Player for Games

def query_player(game, state):
    "Make a move by querying standard input."
    actions = game.actions(state)
    to_move = "Player 1" if game.to_move(state) == 1 else "Player 2"
    if not actions: return None
    while True:
        print(to_move, end=' ')
        move_string = input('move? ')
        try:
            move = eval(move_string)
        except:
            move = move_string
        if move in actions:
            return move

def random_player(game, state):
    "A player that chooses a legal move at random."
    actions = game.actions(state)
    return random.choice(actions) if actions else None

def alphabeta_player(depth=2, eval_fn=eval_fn):
    """A team player that decides after depth plies and evaluates
    the states using the eval_fn function of the team"""
    return (lambda game, state:
            alphabeta_search(state, game, d=depth, eval_fn=eval_fn))

def display_move(player, move):
    "Display a player's move"
    if player == 1:
        print('Player 1', end=' ')
    else:
        print('Player 2', end=' ')
    print("selects", move)

def play_game(game, *players, show=True):
    """Play an n-person, move-alternating game."""
    state = game.initial
    print('INITIAL BOARD')
    game.display(state)
    plays = 0
    while True:  
        plays += 1
        if show: print('PLAY #', plays)
        for player in players:
            if show: print("legal moves=", game.actions(state))
            move = player(game, state)
            if show: display_move(state.to_move, move)
            state = game.result(state, move)
            if game.terminal_test(state):
                print('END BOARD')
                game.display(state)
                utility = game.utility(state, game.to_move(game.initial))
                if utility > 0:
                    print('Player 1 wins!!')
                elif utility < 0:
                    print('Player 2 wins!!')
                else:
                    print('Tie: Nobody wins!!')
                return utility
            if show: game.display(state)

# Examples of calls for playing some games between agents that decide
# using the MINIMAX with alpha-beta pruning, between players that decide
# randomly, and between human players.
# Additionally, in the call you can combine the types of players, change the
# size of the board, and change the depth of the look ahead and the
# evaluation function for the alphabeta_players.
#
#play_game(Knights_Move(), alphabeta_player(1), alphabeta_player(1))
#play_game(Knights_Move(), random_player, random_player, show=False)
#play_game(Knights_Move(), query_player, query_player)
  


In [2]:
play_game(Knights_Move(), alphabeta_player(1, eval_fn = eval_fn_2), alphabeta_player(1, eval_fn = eval_fn))
#play_game(Knights_Move(), random_player, random_player, show=False)

INITIAL BOARD
\ 1 2 3 4 5 6 7 8 /
1 . . . . . . . . 1
2 . . . . . . . . 2
3 . . . . . . . . 3
4 . . . . . . . . 4
5 . . . . . . . . 5
6 . . . . . . . . 6
7 1 . . . 2 . . . 7
8 . . . . . . . . 8
/ 1 2 3 4 5 6 7 8 \
PLAY # 1
legal moves= [(8, 3), (6, 3), (5, 2)]
Evaluation 2:  2.4
Evaluation 2:  5.199999999999999
Evaluation 2:  2.9
Player 1 selects (6, 3)
\ 1 2 3 4 5 6 7 8 /
1 . . . . . . . . 1
2 . . . . . . . . 2
3 . . . . . . . . 3
4 . . . . . . . . 4
5 . . . . . . . . 5
6 . . 1 . . . . . 6
7 _ . . . 2 . . . 7
8 . . . . . . . . 8
/ 1 2 3 4 5 6 7 8 \
legal moves= [(8, 7), (6, 7), (5, 6), (5, 4), (8, 3)]
Evaluation 1:  1.4
Evaluation 1:  3.5
Evaluation 1:  4.8999999999999995
Evaluation 1:  4.8999999999999995
Evaluation 1:  1.4
Player 2 selects (5, 6)
\ 1 2 3 4 5 6 7 8 /
1 . . . . . . . . 1
2 . . . . . . . . 2
3 . . . . . . . . 3
4 . . . . . . . . 4
5 . . . . . 2 . . 5
6 . . 1 . . . . . 6
7 _ . . . _ . . . 7
8 . . . . . . . . 8
/ 1 2 3 4 5 6 7 8 \
PLAY # 2
legal moves= [(8, 2), (8, 4), (5

-1